In [1]:
import json
import os
import glob
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def total_eval_(path):
    model_eval = {'total': {'correct':0, 'incorrect':0, 'total':0, 'unknown':0}}
    
    for num_objects in range(3,11):
        file = open(os.path.join(path, f'val_num_{num_objects}.json'))
        json_object = json.load(file)

        for key in ['yes-no', 'number', 'shape', 'size', 'relation', 'colour', 'material']:
            if key not in json_object.keys():
                continue
            
            model_eval[key] = model_eval.get(key, {'correct':0, 
                                                  'incorrect':0,
                                                  'total':0,
                                                  'unknown':0})
            # print(json_object[key].keys())
            # print(model_eval)
            
            model_eval[key] = {k: model_eval[key][k] + json_object[key][k] for k in json_object[key].keys()}
            # print(model_eval)
            
            model_eval['total'] =  {k: model_eval['total'][k] + json_object[key][k] for k in json_object[key].keys()}
    
    print(model_eval)
        

In [3]:
total_eval_('/scratch/averma90/MLLM_Hallucinations_CLEVR/outputs/baselines/Zephyr_Analysis/InstructBLIP')

{'total': {'correct': 31627, 'incorrect': 40573, 'unknown': 23, 'total': 72223}, 'yes-no': {'correct': 15226, 'incorrect': 14334, 'unknown': 0, 'total': 29560}, 'number': {'correct': 9696, 'incorrect': 14567, 'unknown': 0, 'total': 24263}, 'shape': {'correct': 4080, 'incorrect': 5078, 'unknown': 4, 'total': 9162}, 'colour': {'correct': 2625, 'incorrect': 6594, 'unknown': 19, 'total': 9238}}


In [4]:
total_eval_('/scratch/averma90/MLLM_Hallucinations_CLEVR/outputs/baselines/Zephyr_Analysis/LLaMA')

{'total': {'correct': 36599, 'incorrect': 35606, 'unknown': 18, 'total': 72223}, 'yes-no': {'correct': 15840, 'incorrect': 13720, 'unknown': 0, 'total': 29560}, 'number': {'correct': 15296, 'incorrect': 8967, 'unknown': 0, 'total': 24263}, 'shape': {'correct': 3858, 'incorrect': 5304, 'unknown': 0, 'total': 9162}, 'colour': {'correct': 1605, 'incorrect': 7615, 'unknown': 18, 'total': 9238}}


In [5]:
total_eval_('/scratch/averma90/MLLM_Hallucinations_CLEVR/outputs/baselines/Zephyr_Analysis/LLaVA')

{'total': {'correct': 38269, 'incorrect': 33947, 'unknown': 7, 'total': 72223}, 'yes-no': {'correct': 15113, 'incorrect': 14447, 'unknown': 0, 'total': 29560}, 'number': {'correct': 16009, 'incorrect': 8253, 'unknown': 1, 'total': 24263}, 'shape': {'correct': 3927, 'incorrect': 5235, 'unknown': 0, 'total': 9162}, 'colour': {'correct': 3220, 'incorrect': 6012, 'unknown': 6, 'total': 9238}}


In [1]:
def calculate_precision_recall(tp, fp, tn, fn):
    precision = tp / (tp + fp) if (tp + fp) != 0 else 0
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0
    f1 = 2 * precision * recall / (precision + recall)
    return precision, recall, f1

In [2]:
def precision_recall_lang_aug(path):
    file = open(path)
    json_object = json.load(file)
    yes = {}
    no = {}
    
    # TP = 0
    # FP = 0
    for q_type in ['orig', 'and', 'or_', 'not', 'complex']:
    # for q_type in ['or_']:
        # class yes
        for key in json_object.keys():
            if q_type not in key:
                continue
            if 'yes' in key and '_correct' in key:
                TP = json_object[key]
                # print(TP, key)
            elif 'yes' in key and 'incorrect' in key:
                FN = json_object[key]
            elif 'no' in key and '_correct' in key:
                TN = json_object[key]
            elif 'no' in key and 'incorrect' in key:
                FP = json_object[key]
        # print('CLASS YES', q_type)
        # print(TP, FP, TN, FN)
        yes[q_type] = (calculate_precision_recall(TP, FP, TN, FN)[2])

    
    for q_type in ['orig', 'and', 'or_', 'not', 'complex']:
        # class no
        for key in json_object.keys():
            if q_type not in key:
                continue
            if 'no' in key and '_correct' in key:
                TP = json_object[key]
            elif 'no' in key and 'incorrect' in key:
                FN = json_object[key]
            elif 'yes' in key and '_correct' in key:
                TN = json_object[key]
            elif 'yes' in key and 'incorrect' in key:
                FP = json_object[key]
        # print('CLASS NO', q_type)
        no[q_type] = (calculate_precision_recall(TP, FP, TN, FN)[2])

    print(yes)
    print(no)

In [ ]:
precision_recall_lang_aug('/scratch/averma90/MLLM_Hallucinations_CLEVR/outputs/lang_aug_2/Zephyr_Results/LLaVA')

In [18]:
# precision_recall_lang_aug('/scratch/averma90/MLLM_Hallucinations_CLEVR/outputs/language_augmentation/Zephyr_Analysis/InstructBLIP/new_compiled_results.json')

{'orig': 0.4930593486900345, 'and': 0.619458526588286, 'or_': 0.7207062600321027, 'not': 0.4844471648767458, 'complex': 0.6475431679501459}
{'orig': 0.531448566404958, 'and': 0.2848165416582202, 'or_': 0.4887659868648462, 'not': 0.654668504825166, 'complex': 0.3095929648913815}


In [19]:
# precision_recall_lang_aug('/scratch/averma90/MLLM_Hallucinations_CLEVR/outputs/language_augmentation/Zephyr_Analysis/LLaMA/new_compiled_results.json')

{'orig': 0.5370608591669234, 'and': 0.5774211198249133, 'or_': 0.7594803555462052, 'not': 0.6674301866562697, 'complex': 0.6279066441666904}
{'orig': 0.5383155954682617, 'and': 0.4183750313781273, 'or_': 0.6672971989814478, 'not': 0.48969289019772827, 'complex': 0.3589326631680095}


In [20]:
# precision_recall_lang_aug('/scratch/averma90/MLLM_Hallucinations_CLEVR/outputs/language_augmentation/Zephyr_Analysis/LLaVA/new_compiled_results.json')

{'orig': 0.6576012700230321, 'and': 0.6691503080670583, 'or_': 0.7543667602684309, 'not': 0.6610357241761285, 'complex': 0.6813309685379396}
{'orig': 0.2188047382952825, 'and': 0.07231820008035356, 'or_': 0.560653783944837, 'not': 0.717488789237668, 'complex': 0.24195175639732183}
